# REST Inference

## Setup

Verify that following variable settings match your deployed model's resource name and rest URL. The following code assumes that the kube service is in the same namespace, but you could refer to it in full with the namespace, for example: `http://modelmesh-serving.project-name.svc.cluster.local:8008/v2/models/rafm/infer`

In [ ]:
deployed_model_name = "rafm"
rest_url = "http://modelmesh-serving.rafm:8008"
infer_url = f"{rest_url}/v2/models/{deployed_model_name}/infer"

## Request Function

Build and submit the REST request. 

Note: You submit the data in the same format that you used for an ONNX inference.

In [ ]:
import requests


def rest_request(data):
    json_data = {
        "inputs": [
            {
                "name": "dense_input",
                "shape": [1, 11],
                "datatype": "FP32",
                "data": data
            }
        ]
    }

    response = requests.post(infer_url, json=json_data)
    response_dict = response.json()
    return response_dict['outputs'][0]['data']

In [ ]:
#Load the scaler
import pickle
with open('artifact/scaler.pkl', 'rb') as handle:
    scaler = pickle.load(handle)

## Example 1: Legitimate transaction


In [ ]:
# No-fraud
# Call_Duration 10
# Data_Usage 300
# SMS_Count 5
# Roaming_Indicator 0
# MobileWallet_Use 1
# Cost 50
# Cellular_Location_Distance 3
# Personal_Pin_Used  20
# Avg_Call_Duration 12
# Avg_Data_Usage 350
# Avg_Cost
data = [10,300,5,0,1,50,3,20,12,350,0]
prediction = rest_request(scaler.transform([data]).tolist()[0])
threshhold = 0.95

if (prediction[0] > threshhold):
    print('The model predicts that this is fraud')
else:
    print('The model predicts that this is not fraud')

## Example 2: fraudulent transaction

In [ ]:

data = [300,10000,50,1,1,500,100,1,50,8000,0]

prediction = rest_request(scaler.transform([data]).tolist()[0])
threshhold = 0.95

if (prediction[0] > threshhold):
    print('The model predicts that this is fraud')
else:
    print('The model predicts that this is not fraud')